In [1]:
import numpy as np
from keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras import regularizers
import torch
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import concatenate
from keras import initializers
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU, PReLU
import copy

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
import threading
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pickle
import gc

import keras
import tensorflow as tf

Using TensorFlow backend.


In [8]:
import pickle
with open('/media/data_dump_2/Shivangi/aaai/gossip/finalTrainEmbeddings.pkl', 'rb') as f:
    trainEmbeddings = pickle.load(f)
with open('/media/data_dump_2/Shivangi/aaai/gossip/finalTestEmbeddings.pkl', 'rb') as f:
    testEmbeddings = pickle.load(f)

In [9]:
import json
with open('/media/data_dump_2/Shivangi/aaai/imageDataset/gossip/trainJson.json', 'r') as f:
    trainData = json.load(f)
with open('/media/data_dump_2/Shivangi/aaai/imageDataset/gossip/testJson.json', 'r') as f:
    testData = json.load(f)

In [10]:
for i in trainEmbeddings:
    if len(trainEmbeddings[i]) >=50:
        trainEmbeddings[i] = trainEmbeddings[i][0:50]
    else:
        deficit = 50 - len(trainEmbeddings[i])
        for j in range(deficit):
            trainEmbeddings[i].append(np.zeros((1,768), dtype=np.float32))
    temp = np.empty((50,768), dtype=np.float32)
    for j in range(len(trainEmbeddings[i])):
        temp[j][:] = trainEmbeddings[i][j]
    trainEmbeddings[i] = temp

In [11]:
with open('/media/data_dump_2/Shivangi/aaai/gossip/train_gossip.pickle', 'rb') as f:
    train_vgg_poli = pickle.load(f)
with open('/media/data_dump_2/Shivangi/aaai/gossip/test_gossip.pickle', 'rb') as f:
    test_vgg_poli = pickle.load(f)

In [12]:
for i in testEmbeddings:
    if len(testEmbeddings[i]) >=50:
        testEmbeddings[i] = testEmbeddings[i][0:50]
    else:
        deficit = 50 - len(testEmbeddings[i])
        for j in range(deficit):
            testEmbeddings[i].append(np.zeros((1,768), dtype=np.float32))
    temp = np.empty((50,768), dtype=np.float32)
    for j in range(len(testEmbeddings[i])):
        temp[j][:] = testEmbeddings[i][j]
    testEmbeddings[i] = temp

In [13]:
train_text = []
train_label = []
test_text = []
test_label = []
train_image = []
test_image = []

In [14]:
for i in train_vgg_poli:
    for j in trainData:
        if i.split('.jpg')[0] == trainData[j][-1]['Top_img']:
            if j in trainEmbeddings:
                train_text.append(trainEmbeddings[j])
                train_image.append(train_vgg_poli[trainData[j][-1]['Top_img'] + '.jpg'])
                train_label.append(trainData[j][0])

In [15]:
testEmbedOrder = []
testImageOrder = []

In [16]:
for i in test_vgg_poli:
    for j in testData:
        if i.split('.jpg')[0] == testData[j][-1]['Top_img']:
            if j in testEmbeddings:
                test_text.append(testEmbeddings[j])
                test_image.append(test_vgg_poli[testData[j][-1]['Top_img'] + '.jpg'])
                test_label.append(testData[j][0])
                testEmbedOrder.append(j)
                testImageOrder.append(i)

In [17]:
2458

2458

In [18]:
len(train_text), len(train_image), len(train_label)

(10010, 10010, 10010)

In [19]:
len(test_text), len(test_image), len(test_label), len(testEmbedOrder), len(testImageOrder)

(2830, 2830, 2830, 2830, 2830)

In [20]:
realCount = 0
fakeCount = 0
for i in train_label:
    if i == 1:
        realCount += 1
    elif i == 0:
        fakeCount += 1
for i in test_label:
    if i == 1:
        realCount += 1
    elif i == 0:
        fakeCount += 1
print(realCount, fakeCount)

10259 2581


In [14]:
discreteTest = copy.deepcopy(test_label)2458

In [15]:
train_label = to_categorical(train_label)
test_label = to_categorical(test_label)

In [16]:
"""train_text=[torch.Tensor.numpy(i.cpu()) for i in train_text]
test_text=[torch.Tensor.numpy(i.cpu()) for i in test_text]"""
train_image[0].shape

(1, 4096)

In [17]:
train_text_matrix = np.ndarray(shape=(len(train_text), 50,768))

In [18]:
counter = 0
for i in train_text:
    train_text_matrix[counter][:][:] = i
    counter += 1

In [19]:
test_text_matrix = np.ndarray(shape=(len(test_text), 50,768))

In [20]:
counter = 0
for i in test_text:
    test_text_matrix[counter][:][:] = i
    counter += 1

In [21]:
test_text_matrix.shape

(2830, 50, 768)

In [22]:
train_image[0].shape

(1, 4096)

In [23]:
train_image_matrix = np.ndarray(shape=(len(train_image), 4096,1))

In [24]:
counter = 0
for i in train_image:
    train_image_matrix[counter][:][:] = i.reshape(4096,1)
    counter += 1

In [25]:
test_image_matrix = np.ndarray(shape=(len(test_image), 4096,1))

In [26]:
counter = 0
for i in test_image:
    test_image_matrix[counter][:][:] = i.reshape(4096,1)
    counter += 1

In [27]:
len(train_image_matrix)

10010

In [28]:
train_image_matrix = train_image_matrix.reshape(len(train_image_matrix),4096)
test_image_matrix = test_image_matrix.reshape(len(test_image_matrix),4096)

## Inference of MM model

In [44]:
from keras.models import load_model
model = load_model('checkpoints_gossip/dense_Text_model.hdf5')

In [45]:
predictions = model.predict(test_text_matrix)
predictions = predictions.argmax(axis=1)

In [90]:
data = []

In [91]:
for i in range(len(test_label)):
    data.append([testEmbedOrder[i],testImageOrder[i],discreteTest[i],predictions[i]])

In [92]:
from pandas import DataFrame
df = DataFrame.from_records(data)

In [94]:
df

,0,1,2,3
0,JR Smith's Daughter Released from NICU After B...,01r40J3oClyYJCLDrlzSYJ5c69lHOMux.jpg,1,1
1,Sofia Richie Has a Scott Disick Phone Case and...,01whK2ttpeODIWZrvYFkowXXjyaDrZPs.jpg,1,1
2,Everything We Know About Brad Pitt’s Plane Inc...,02s3CNoDOz8F0oeX3tA74g6CRpmxHnvc.jpg,0,0
3,Kate Middleton Secretly Donated Her 7 Inches o...,05LFsMa2ZHO6z2GLQzaLeI8YBxCkFufq.jpg,1,1
4,General Gabbery: DWTS Or sign in with one of t...,07bXihvPVrtYL6NWtmBLRocPl6kCtGQr.jpg,1,1
...,...,...,...,...
2825,Francesca Eastwood Is Pregnant! Daughter of Cl...,A0w05t3M4mDp1k78mLZxmpgA737vbwgs.jpg,1,1
2826,"Stevie Ryan Stevie Kathleen Ryan (June 2, 1984...",a0wF3r18U9Pe2YEJC9S1BBQgsHmCiDdu.jpg,1,1
2827,Oprah tried her first Auntie Anne's pretzel an...,A38RDaGDi7RGDYpzTDfT1WIZYtj6gh1K.jpg,1,1
2828,Demi Lovato on Her Wild Party with Snoop Dogg:...,a3fQ8ZsAZ0YRefUhOeGchznK5vJbm4VW.jpg,1,1


In [95]:
df.to_csv('gossip_text.csv', sep='\t')
df.columns = ["Text", "image" ,"true", "predicted"]

In [128]:
df.iloc[9]['Text']

'Kim Kardashian, Kanye West Snubbed By Kate Middleton And Prince William Report Debunked Kate Middleton and Prince William are not spared from rumors and malicious reports despite their stature. But the rumors of them snubbing Kim Kardashian and Kanye West was debunked. (Photo: Chris Jackson/Getty Images)\n\nKate Middleton and Prince William despite their stature in the world are not spared from rumors and unfounded reports that put the royal couple in a bad light. One of them is the report that they snubbed the power couple Kim Kardashian and Kanye West.\n\nThe report said that Kardashian feels that Middleton and Prince William are snubbing or disinterested in meeting her and her husband. The royal couple allegedly turned down the invitation to attend West\'s concert a couple of times. The "Keeping up with the Kardashians" star feels that it is snobbery towards her.\n\nThe mother of two was not in good terms with Beyonce. She knew that the royal couple met with her during a Nets game 

In [110]:
df.loc[df['true'] < df['predicted']]

,Text,image,true,predicted
9,"Kim Kardashian, Kanye West Snubbed By Kate Mid...",0CmVkvav787PZg99LAOi4w1gO7STt95a.jpg,0,1
18,Charlie Sheen says he is HIV-positive (CNN) Ac...,MTSvnGpgyCRAxMysfxPE0ZV7CZapnc0R.jpg,0,1
21,Duchess Meghan and Prince Harry Are Planning a...,MUvYxKV9Qj7zpDuFObuvSSLt6JVO8pXn.jpg,0,1
29,‘Fixer Upper’ Gets November Premiere Date For ...,mW6itzHKfU990NTOLG5KikJG0j7ml6FX.jpg,0,1
30,Patrick Dempsey: Why I Called Off My Divorce t...,etRcWhpIZZ2SARMBXfSEXZQfDvwUguB2.jpg,0,1
...,...,...,...,...
2796,Kim Kardashian Having Plastic Surgery To Cure ...,yjT2Y9mBErC1UubMFJYo1MtZSRnjKU0I.jpg,0,1
2799,Kelly and Ryan’s relationship may not be as su...,yKHF16O9pcHS3zdoCIPqt85dr37VbJtu.jpg,0,1
2810,Christina Aguilera Joining ‘American Idol’?: A...,JzJ3HynrlaxsnQ74U8qrFp9fMVOVFsjv.jpg,0,1
2814,Jennifer Aniston Plastic Surgery- Celebrity Co...,LQXtiIkANvlfUUlpFmE9zZ0yBhhqpDlg.jpg,0,1


In [79]:
import pandas as pd
df_mm = pd.read_csv('gossip_mm.csv', sep='\t')
df_mm.columns = ["Index", "Text", "image" ,"true", "predicted"]

In [116]:
#df_mm.loc[df_mm['true'] > df_mm['predicted']]
df_mm.loc[2815]

Index                                                     2815
Text         Robert Pattinson and FKA twigs Might Not Marry...
image                     Lw3FNiAoy5GdIJ4DVO39YbQXlFljZ0gB.jpg
true                                                         0
predicted                                                    0
Name: 2815, dtype: object

In [ ]:
#  Indices for 1-1: 2157, 2458,2558